# From spreadsheet to Kobo
---

![](https://media.giphy.com/media/xUNd9KMFMPtn5reCiI/giphy.gif)

Sometimes you might want to import data from a spreadsheet into the Kobo database. Maybe you've exported your original data into a spreadsheet, done some data-cleaning and now you want it back. If that's the case, you may want to checkout the [bulk updating feature](https://support.kobotoolbox.org/howto_edit_multiple_submissions.html) now included in the UI. Whatever the case, perhaps the following tutorial may help — or at least guide you in a right direction.

## Get things setup
---

You may need to `pip install` some necessary packages to perform some functions depending on your local setup. I'm currently using the [`jupyter/datascience-notebook`](https://registry.hub.docker.com/r/jupyter/datascience-notebook/#!) Docker image. If you have Docker installed locally, you can do the same by running the following command and pasting the generated URL into your browser:

```bash
docker run -p 8888:8888 -v ~/notebooks:/home/jovyan jupyter/datascience-notebook
```

Once that's out the way, let's go ahead and import all that we need.

### Install any necessary packages ⬇️

In [ ]:
!pip install openpyxl

### Import whatever you need 👇

In [4]:
import io
import requests
import uuid
from datetime import datetime
from random import choice, randint, sample
from time import sleep
from xml.etree import ElementTree as ET

import pandas as pd
import pytz

### Set up some helpful constants that we'll use below 🤘

In [16]:
TOKEN='90ef6878cea96288f146c490bee694f3d3de5ad3'

KC_URL = 'https://ktc.kobocaid.eu'
KF_URL = 'https://ktf.kobocaid.eu'

ASSET_UID = 'akEPXAsUX5E5FXhqsnJDfk'

DATA_URL = f'{KF_URL}/api/v2/assets/{ASSET_UID}/data'
XML_URL = f'{DATA_URL}.xml'
SUMISSION_URL = f'{KC_URL}/api/v1/submissions'

HEADERS = {
    'Authorization': f'Token {TOKEN}'
}
PARAMS = {
    'format': 'json'
}

## Let's create some random submission data 🎉

![](https://media.giphy.com/media/xT9C25UNTwfZuk85WP/giphy.gif)

For the purposes of demonstration, let's generate a list of submissions with random names and random choices from the available list of pizza toppings: cheese, pepperoni and saussage.

Those submissions will then be exported into an Excel file so that we have what we need to continue with the walkthrough of submitting data from an Excel spreadsheet.

In [17]:
res = requests.get(url=XML_URL, headers=HEADERS, params=PARAMS)

In [18]:
res.status_code

200

If you take a look at the XML structure, the submissions are nested within `results` and use the `ASSET_UID` as their root.

In [19]:
parsed_xml = ET.fromstring(res.text)
e = parsed_xml.find(f'results/{ASSET_UID}')
template = ET.tostring(e)

In [20]:
print(template.decode())

<akEPXAsUX5E5FXhqsnJDfk id="akEPXAsUX5E5FXhqsnJDfk" version="1 (2025-03-07 11:30:57)">&lt;?xml version='1.0' encoding='UTF-8' ?&gt;&lt;aDAE5k6arKDTPboTwLFSb8 id="aDAE5k6arKDTPboTwLFSb8" version="vVNzMLHHMuYB9FHNtoWk5F" xmlns:ev="http://www.w3.org/2001/xml-events" xmlns:orx="http://openrosa.org/xforms" xmlns:odk="http://www.opendatakit.org/xforms" xmlns:h="http://www.w3.org/1999/xhtml" xmlns:jr="http://openrosa.org/javarosa" xmlns:xsd="http://www.w3.org/2001/XMLSchema"&gt;&lt;formhub&gt;&lt;uuid&gt;20178d240cd14048b9eca2fc1fd591dc&lt;/uuid&gt;&lt;/formhub&gt;&lt;start&gt;2024-12-02T12:03:52.766+03:00&lt;/start&gt;&lt;end&gt;2024-12-02T12:56:26.619+03:00&lt;/end&gt;&lt;Organisation&gt;aph&lt;/Organisation&gt;&lt;Activity&gt;mhpss&lt;/Activity&gt;&lt;Hello_what_is_your_eaking_with_me_today&gt;yes&lt;/Hello_what_is_your_eaking_with_me_today&gt;&lt;Do_you_give_consent_to_take_part&gt;yes&lt;/Do_you_give_consent_to_take_part&gt;&lt;Can_you_tell_me_a_bi_you_been_involved_in&gt;Resin and now p

### Time for some hacking 💻

![](https://media.giphy.com/media/wpoLqr5FT1sY0/giphy.gif)

Now that we've got a template, let's go ahead and create some helpful methods to parse and update that template with the data in the Excel spreadsheet.

(If you're interested, most of the process below is adapted from the code in KPI that interfaces with Kobocat [here](https://github.com/kobotoolbox/kpi/blob/d56621b6daced1891bc9fe2661c2aafe9e9a92a4/kpi/deployment_backends/kobocat_backend.py#L641-L722).)

In [ ]:
def submit_data(xml_sub: bytes, _uuid: str) -> str:
    """
    Send the XML to kobo!
    """
    file_tuple = (_uuid, io.BytesIO(xml_sub))
    files = {'xml_submission_file': file_tuple}
    res = requests.Request(
        method='POST', url=SUMISSION_URL, files=files, headers=HEADERS
    )
    session = requests.Session()
    res = session.send(res.prepare())
    return res.status_code

def format_openrosa_datetime() -> str:
    """
    This is required to get the correct datetime formatting
    """
    return datetime.now(tz=pytz.UTC).isoformat('T', 'milliseconds')

def update_element_value(e: ET.Element, name: str, value: str) -> None:
    """
    Get or create a node and give it a value
    """
    el = e.find(name)
    if el is None:
        el = ET.SubElement(e, name)
    el.text = value

def create_submissions(data: pd.DataFrame) -> list:
    """
    Take a bunch of submissions and send them off
    """
    all_subs = []
    parsed_xml = ET.fromstring(template)
    for i, row in data.iterrows():
        _now = format_openrosa_datetime()
        _uuid = str(uuid.uuid4())

        for item in data.columns:
            update_element_value(parsed_xml, item, row[item])

        # We have to update the instanceID, otherwise there'll be issues
        update_element_value(parsed_xml, 'meta/instanceID', f'uuid:{_uuid}')

        # Updating the `start` and `end` times is not really necessary, but
        # probably something you'd want to do
        update_element_value(parsed_xml, 'start', _now)
        update_element_value(parsed_xml, 'end', _now)

        all_subs.append(submit_data(ET.tostring(parsed_xml), _uuid))

        # If you are submitting a large amount of data, please be mindful that it can
        # overwhelm the servers if sent in a short span of time. Letting it sleep for
        # for a short stint between each upload will be much appreciated
        sleep(0.2)

    return all_subs

In [ ]:
responses = create_submissions(subs)

Let's do a quick check to see if all the responses were successful:

In [ ]:
all(res == 201 for res in responses)

True

If there were a few unsuccessful, let's see how many...

In [ ]:
pd.Series(responses).value_counts()

201    20
dtype: int64

## All done 🔥

Check the data table in the Kobo UI and ensure that your data has been successfully submitted.

![](https://media.giphy.com/media/lD76yTC5zxZPG/giphy.gif)